# Import libraries

For openai>=1.0.0, openai.ChatCompletion is not supported anymore. Install an earlier version. Install cohere and tiktoken to address the Error message that recently came up:
<blockquote>ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.<br>
llmx 0.0.15a0 requires cohere, which is not installed.<br>
llmx 0.0.15a0 requires tiktoken, which is not installed.</blockquote>

In [ ]:
!pip install openai==0.27.8 cohere tiktoken
!pip install python-dotenv

In [ ]:
import inspect
import os
import dotenv
import openai
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/datasets/IHC_classes_gpt-3.5-turbo-0301',
            exist_ok=True)
os.makedirs('/content/drive/MyDrive/datasets/IHC_classes_gpt-3.5-turbo-0613',
            exist_ok=True)

# Authentication

You have to first save your OpenAI API key in a .env file in your Google Drive, in the format:
OPENAI_API_KEY = '...'.

In [ ]:
# OpenAI authentication on Google Colab. via
# https://stackoverflow.com/a/77166086
dotenv.load_dotenv('/content/drive/MyDrive/.env')
openai.api_key = os.getenv('OPENAI_API_KEY')

# Import dataset

In [ ]:
implicit_hate_stg1_posts = pd.read_csv(
    '/content/drive/MyDrive/datasets/implicit_hate_v1_stg1_posts.tsv',
    sep='\t')
implicit_hate_stg1_posts = implicit_hate_stg1_posts.sample(n=2685,
                                                           random_state=1)
implicit_hate_stg1_posts = implicit_hate_stg1_posts.reset_index(drop=True)

# Set prompts

In [ ]:
def system_content(persona):
    return inspect.cleandoc(
        f"""From now on, act as {persona}.
        Provide an answer that {persona} would regarding the text.""")

In [ ]:
def user_content(tweet):
    return inspect.cleandoc(f"""Text: \"\"\"
    {tweet}
    \"\"\"

    Does this text attack a person or a group of people, explicitly or
    implicitly, based on their protected characteristics?

    A: Yes, explicit hate speech
    B: Yes, implicit hate speech
    C: Not hate speech

    Constraint: Answer with only the option above that is most accurate and
    nothing else.

    Tip: Explicit hate speech uses explicit hatred expressions or offensive
    keywords such as n*gger, c*nt, etc. to attack a person or a group of
    people based on their protected characteristics. Implicit hate speech is a
    more complex attack on a person or a group of people without relying on
    explicit keywords. Beware that implicit discriminatory speech may be
    subtle. Protected characteristics include ethnicity, race, national origin,
    religion, sex, gender, and sexual orientation. Note that pointing out
    racism should not be considered as hate speech.""")

# Define a function to call Chat Completion API with (persona) or without (no-persona default) the *system message*

In [ ]:
# Prevent rate limit errors. via
# https://github.com/openai/openai-cookbook/blob/main/examples/
# How_to_handle_rate_limits.ipynb
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def classifier_with_backoff(user_cont, model, system_cont=None):
    if system_cont is None:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_cont}
            ],
            temperature=0,
        )
    else:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": system_cont},
                {"role": "user", "content": user_cont}
            ],
            temperature=0,
        )
    return response['choices'][0]['message']['content']

# Define a function to perform model inference on **IHC**

Create a function `predict_high_level_classes` that takes a list of personas (e.g., ['a White person', 'a Black or African American']), a `topic` (e.g., a sociodemographic attribute such as *Race/Ethnicity*), and a GPT model name (e.g., gpt-3.5-turbo-0301) as required input arguments. The optional argument `base` determines if annotations made by the no-persona default should be included. For each persona in the list of personas, the function iterates over each tweet in **IHC**, calling the function `classifier_with_backoff` for each tweet.  It then adds the respective lists of classes predicted by each simulated persona as new columns to the **IHC** dataframe. The predicted classes are remapped to conform to how the data was originally coded in **IHC**, e.g., `explicit_hate` if `A: Yes, explicit hate speech`. The dataframe is saved in TSV file format, and the function returns the dataframe.

In [ ]:
def predict_high_level_classes(persona_list, topic, model, base=None):
    df = implicit_hate_stg1_posts.copy(deep=True)
    tweets = df['post'].values
    if base == 'Yes':
        df['predicted_classes_base'] = [
            classifier_with_backoff(user_content(twt),
                                    model) for twt in tweets]
    for persona in persona_list:
        persona += ' in the United States'
        df[f'predicted_classes_{persona}'.replace(' ', '_')] = [
            classifier_with_backoff(user_content(twt),
                                    model,
                                    system_content(persona)) for twt in tweets]
    # Remap predicted class labels. via
    # https://stackoverflow.com/a/20250996
    remap = {'A: Yes, explicit hate speech': 'explicit_hate',
             'B: Yes, implicit hate speech': 'implicit_hate',
             'C: Not hate speech': 'not_hate'}
    for i in df.iloc[:, 2:].columns:
        df.replace({i: remap}, inplace=True)
    df.to_csv(
        f'/content/drive/MyDrive/datasets/IHC_classes_{model}/'
        f'IHC_classes_predictions_{topic}_{model}.tsv',
        sep='\t', index=False)
    return df

# Create dictionaries, each with a `topic` as the key and a list of personas as the value

In [ ]:
gender_genderidentity_sexuality = {
    'gender_genderidentity_sexuality': ['a man',
                                        'a woman',
                                        'a non-binary person',
                                        'a LGBTQIA+ person']}

In [ ]:
religion = {'religion': ['a Muslim',
                         'an atheist',
                         'a Buddhist',
                         'a Christian',
                         'a Hindu',
                         'a Jew']}

In [ ]:
race_ethnicity = {
    'race_ethnicity': ['a White person',
                       'a Black or African American',
                       'an American Indian or Alaska Native',
                       'an Asian',
                       'a Native Hawaiian or Other Pacific Islander',
                       'a Hispanic/Latinx person']}

In [ ]:
origin = {'origin': ['an immigrant',
                     'a refugee',
                     'an American citizen']}

# Call the `predict_high_level_classes` function

Running each of the following cells could take several hours. Colab Pro+ includes continuous code execution (capped at 24 hours) and background execution capabilities, enabling you to close your browser/device while your code runs. CPU is sufficient. Alternatively, you can download the notebooks and edit the code accordingly to point to the directories on your local machine you want to use.

In [ ]:
predict_high_level_classes(list(gender_genderidentity_sexuality.values())[0],
                           list(gender_genderidentity_sexuality.keys())[0],
                           'gpt-3.5-turbo-0301',
                           'Yes')

In [ ]:
predict_high_level_classes(list(gender_genderidentity_sexuality.values())[0],
                           list(gender_genderidentity_sexuality.keys())[0],
                           'gpt-3.5-turbo-0613',
                           'Yes')

In [ ]:
predict_high_level_classes(list(religion.values())[0],
                           list(religion.keys())[0],
                           'gpt-3.5-turbo-0301')

In [ ]:
predict_high_level_classes(list(religion.values())[0],
                           list(religion.keys())[0],
                           'gpt-3.5-turbo-0613')

In [ ]:
predict_high_level_classes(list(race_ethnicity.values())[0],
                           list(race_ethnicity.keys())[0],
                           'gpt-3.5-turbo-0301')

In [ ]:
predict_high_level_classes(list(race_ethnicity.values())[0],
                           list(race_ethnicity.keys())[0],
                           'gpt-3.5-turbo-0613')

In [ ]:
predict_high_level_classes(list(origin.values())[0],
                           list(origin.keys())[0],
                           'gpt-3.5-turbo-0301')

In [ ]:
predict_high_level_classes(list(origin.values())[0],
                           list(origin.keys())[0],
                           'gpt-3.5-turbo-0613')